In [ ]:
def having_ip_address(url):
  import re
  x = re.search('^(http|https)://\d+\.\d+\.\d+\.\d+\.*', url)
  if x:
    return 1
  else:
    return -1

In [ ]:
having_ip_address('https://128.36.54.192/questions/59020008/how-to-import-functions-of-a-jupyter-notebook-into-another-jupyter-notebook-in-g')

In [ ]:
def URL_Length(url):
  if len(url) < 54:
    return -1
  elif len(url) >= 54 and len(url) <=75:
    return 0
  else:
    return 1

In [ ]:
URL_Length('https://stackoverflow.com/questions/59020008/how-to-import-functions-of-a-jupyter-notebook-into-another-jupyter-notebook-in-g')

In [ ]:
#Module 3

In [ ]:
def haveAtSign(url):
  if "@" in url:
    at = 1    
  else:
    at = -1    
  return at

In [ ]:
haveAtSign('https://stackoverflow.com/questions/59020008/how-to-import-functions-of-a-jupyter-notebook-into-another-jupyter-notebook-in-g')

In [ ]:
#Prefix_Suffix

In [ ]:
def prefixSuffix(url):
    from urllib.parse import urlparse
    if '-' in urlparse(url).netloc:
        return 1
    else:
        return -1

In [ ]:
prefixSuffix('https://dfhdfhdfgs.tokyo/ja-jp/account/login')

In [ ]:
#Having Sub Domain

In [ ]:
def sub_domain_count(url):
  !pip install tld
  from urllib.parse import urlparse
  from tld import get_tld
  domain = urlparse(url).netloc
  domain = domain.split('.')
  top_domain = get_tld(url)
  top_domain = top_domain.split('.')
  sub_domain = set(domain) - set(top_domain)
  count = len(sub_domain)
  if(count == 1):
    return -1
  if(count == 2):
    return 0
  else:
    return 1 

In [ ]:
sub_domain_count('https://stackoverflow.com/questions/59020008/how-to-import-functions-of-a-jupyter-notebook-into-another-jupyter-notebook-in-g')

In [ ]:
###SSL final state

In [ ]:
def sslVerify(url):
  from urllib.parse import urlparse,urlencode,urlsplit, urlunsplit
  from urllib.request import Request, urlopen, ssl, socket
  import json
  from datetime import datetime
  split_url = urlsplit(url)
  #some site without http/https in the path
  port = '443'

  hostname = split_url.netloc
  context = ssl.create_default_context()
  try:
    with socket.create_connection((hostname, port)) as sock:
      with context.wrap_socket(sock, server_hostname=hostname) as ssock:
          data = json.dumps(ssock.getpeercert())
          res = json.loads(data)
    notBefore = datetime.strptime(res["notBefore"],'%b  %d %H:%M:%S %Y %Z').date()
    notAfter = datetime.strptime(res["notAfter"],'%b  %d %H:%M:%S %Y %Z').date()
    # print(notBefore, notAfter)
    if(ssl.SSLCertVerificationError(ssock) == True):
      return 1
    elif (notAfter.year-notBefore.year)+(notAfter.month-notBefore.month)*0.1 >= 1:
      return -1
    else:
      return 0
  except:
    return 1

In [ ]:
sslVerify("http://postdebanks.com/DIE/POST/diepost/")

In [ ]:
#port

In [ ]:
def port(domain):
  import requests
  import json
  response = requests.get("https://api.viewdns.info/portscan/?host="+domain+"&apikey=1bf03196763a201bcc66a59bf88ed8ddf7a9432f&output=json")
  myjson = response.json()
  # print(myjson)
  pref_stat = {21:'closed', 22:'closed', 23:'closed', 80:'open', 443:'open', 445 : 'closed', 1433:'closed', 1521:'closed', 3306 : 'closed', 3389:'closed' }
  flag = -1
  for i in range(len(myjson['response']['port'])):
    if int(myjson['response']['port'][i]['number']) in pref_stat:
      if(myjson['response']['port'][0]['status'] != pref_stat[int(myjson['response']['port'][0]['number'])]):
        flag = 1
  return flag

In [ ]:
# port('postdebanks.com')

In [ ]:
#Request_URL

In [ ]:
def request_url(url):
  from urllib.parse import urlparse,urlencode,urlsplit, urlunsplit
  from bs4 import BeautifulSoup
  import requests
  import re
  !pip install tldextract
  import tldextract
  Null_format = ["", "#", "#nothing", "#doesnotexist", "#null", "#void", "#whatever",
               "#content", "javascript::void(0)", "javascript::void(0);", "javascript::;", "javascript"]
  
  def is_URL_accessible(url):
    page = None
    try:
        page = requests.get(url, timeout=5)   
    except:
        parsed = urlparse(url)
        url = parsed.scheme+'://'+parsed.netloc
        if not parsed.netloc.startswith('www'):
            url = parsed.scheme+'://www.'+parsed.netloc
            try:
                page = requests.get(url, timeout=5)
            except:
                page = None
                pass
    if page and page.status_code == 200 and page.content not in ["b''", "b' '"]:
        return True, url, page
    else:
        return False, None, None
    
  state, iurl, page = is_URL_accessible(url)

  def get_domain(url):
      o = urlsplit(url)
      return o.hostname, tldextract.extract(url).domain, o.path
    
  if state:
        print('Yes')
        content = page.content
        hostname, domain, path = get_domain(url)
  else:
        print('No state')
  
  Media = {'internals':[], 'externals':[], 'null':[]}

  def external_media(Media):
    total = len(Media['internals']) + len(Media['externals'])
    externals = len(Media['externals'])
    try:
        percentile = externals / float(total) * 100
    except:
        return 0
    return percentile
  
  def findMedia(Media,domain, hostname ):
    Null_format = ["", "#", "#nothing", "#doesnotexist", "#null", "#void", "#whatever",
               "#content", "javascript::void(0)", "javascript::void(0);", "javascript::;", "javascript"]
    soup = BeautifulSoup(content, 'html.parser', from_encoding='iso-8859-1')
    for img in soup.find_all('img', src=True):
          dots = [x.start(0) for x in re.finditer('\.', img['src'])]
          if hostname in img['src'] or domain in img['src'] or len(dots) == 1 or not img['src'].startswith('http'):
              if not img['src'].startswith('http'):
                  if not img['src'].startswith('/'):
                      Media['internals'].append(hostname+'/'+img['src']) 
                  elif img['src'] in Null_format:
                      Media['null'].append(img['src'])  
                  else:
                      Media['internals'].append(hostname+img['src'])   
          else:
              Media['externals'].append(img['src'])
    for audio in soup.find_all('audio', src=True):
      dots = [x.start(0) for x in re.finditer('\.', audio['src'])]
      if hostname in audio['src'] or domain in audio['src'] or len(dots) == 1 or not audio['src'].startswith('http'):
          if not audio['src'].startswith('http'):
              if not audio['src'].startswith('/'):
                  Media['internals'].append(hostname+'/'+audio['src']) 
              elif audio['src'] in Null_format:
                  Media['null'].append(audio['src'])  
              else:
                  Media['internals'].append(hostname+audio['src'])   
      else:
          Media['externals'].append(audio['src'])
          
    for embed in soup.find_all('embed', src=True):
      dots = [x.start(0) for x in re.finditer('\.', embed['src'])]
      if hostname in embed['src'] or domain in embed['src'] or len(dots) == 1 or not embed['src'].startswith('http'):
          if not embed['src'].startswith('http'):
              if not embed['src'].startswith('/'):
                  Media['internals'].append(hostname+'/'+embed['src']) 
              elif embed['src'] in Null_format:
                  Media['null'].append(embed['src'])  
              else:
                  Media['internals'].append(hostname+embed['src'])   
      else:
          Media['externals'].append(embed['src'])
        
    for i_frame in soup.find_all('iframe', src=True):
      dots = [x.start(0) for x in re.finditer('\.', i_frame['src'])]
      if hostname in i_frame['src'] or domain in i_frame['src'] or len(dots) == 1 or not i_frame['src'].startswith('http'):
          if not i_frame['src'].startswith('http'):
              if not i_frame['src'].startswith('/'):
                  Media['internals'].append(hostname+'/'+i_frame['src']) 
              elif i_frame['src'] in Null_format:
                  Media['null'].append(i_frame['src'])  
              else:
                  Media['internals'].append(hostname+i_frame['src'])   
      else: 
          Media['externals'].append(i_frame['src'])
  
  findMedia(Media, domain, hostname)
  if external_media(Media) < 22:
    return -1
  elif external_media(Media) >= 22 and external_media(Media) < 61:
    return 0
  else:
    return 1

In [ ]:
# request_url('http://www.budgetbots.com/server.php/Server%20update/index.php?email=USER@DOMAIN.com')

In [ ]:
#Safe anchors

In [ ]:
def url_anchor(url):
  from urllib.parse import urlparse,urlencode,urlsplit, urlunsplit
  from bs4 import BeautifulSoup
  import requests
  import re
  !pip install tldextract
  import tldextract
  Null_format = ["", "#", "#nothing", "#doesnotexist", "#null", "#void", "#whatever",
               "#content", "javascript::void(0)", "javascript::void(0);", "javascript::;", "javascript"]
  
  def is_URL_accessible(url):
    page = None
    try:
        page = requests.get(url, timeout=5)   
    except:
        parsed = urlparse(url)
        url = parsed.scheme+'://'+parsed.netloc
        if not parsed.netloc.startswith('www'):
            url = parsed.scheme+'://www.'+parsed.netloc
            try:
                page = requests.get(url, timeout=5)
            except:
                page = None
                pass
    if page and page.status_code == 200 and page.content not in ["b''", "b' '"]:
        return True, url, page
    else:
        return False, None, None
    
  state, iurl, page = is_URL_accessible(url)

  def get_domain(url):
      o = urlsplit(url)
      return o.hostname, tldextract.extract(url).domain, o.path
    
  if state:
        content = page.content
        hostname, domain, path = get_domain(url)
    
  Anchor = {'safe':[], 'unsafe':[], 'null':[]}

  def anchor(Anchor, domain, hostname):
      soup = BeautifulSoup(content, 'html.parser', from_encoding='iso-8859-1')
      for href in soup.find_all('a', href=True):
        dots = [x.start(0) for x in re.finditer('\.', href['href'])]
        if hostname in href['href'] or domain in href['href'] or len(dots) == 1 or not href['href'].startswith('http'):
            if "#" in href['href'] or "javascript" in href['href'].lower() or "mailto" in href['href'].lower():
                 Anchor['unsafe'].append(href['href'])
        else:
            Anchor['safe'].append(href['href'])

  anchor(Anchor, domain, url)

  def safe_anchor(Anchor):
      total = len(Anchor['safe']) +  len(Anchor['unsafe'])
      unsafe = len(Anchor['unsafe'])
      try:
        percentile = unsafe / float(total) * 100
      except:
        return 0
      return percentile
  #print(safe_anchor(Anchor))   
  if safe_anchor(Anchor) < 31:
       return -1
  elif safe_anchor(Anchor) >= 31 and safe_anchor(Anchor) <= 67:
       return 0
  else:
       return 1

In [ ]:
url_anchor('https://www.xiaoji.com/')